# Time Series analysis of phase in SWA regions for SWH and WSP 

Plot figure within jupyter notebook

In [1]:
%matplotlib inline

Use the sys library in order to tell the notebook to look for files within the followinf directory path: 

In [2]:
import sys
sys.path.append('/zdata/home/lcolosi/python_functions/')

Import all libraries and functions

In [3]:
#libraries
import numpy as np #contains the major of functions used for matrix arrays  
import matplotlib.pyplot as plt # matplotlib contains functions for graphics and plot manipulation
from netCDF4 import Dataset, num2date # netCDF4 handles netCDF files 
import glob 
import cmocean.cm as cmo
from matplotlib import cm 

#my functions
from running_mean import running_mean
from detrend import detrend
from unweighted_least_square_fit import least_square_fit 
from char_LSF_curve import character_LSF
import cartopy_fig_module as cart

Set dimensions for data of space and time which depends on the spatial orientation of the data set and the time period which the data is collected from. In addition, pick the resolution of the grid boxes which the climatologies will be computed from: 

In [4]:
nt = 8400 
nlon, nlat = 90, 34
nlon_i, nlat_i = 360, 133
nlon_c, nlat_c = 1440, 529
ngrid = 3
month = np.arange(1,13,1)

Set Ifremer filename and look at key variables and attributes

In [5]:
filename_i = '/zdata/downloads/colosi_data_bk/binned_data/ifremer_p1_daily_data/bia_daily_binned_ifremer_data/all_sat_binned_swh_1992-08-23_2016-08-23.nc'

#set nc variable in order to read attributes and obtained data: 
nc_i = Dataset(filename_i, 'r')

#print key variables:
print(nc_i.variables.keys())

#longitude
for at in nc_i.variables['lon'].ncattrs():
    print("%s : %s" %(at, nc_i.variables['lon'].getncattr(at)))

#laitude
for at in nc_i.variables['lat'].ncattrs():
    print("%s : %s" %(at, nc_i.variables['lat'].getncattr(at)))
    
#time 
for at in nc_i.variables['time'].ncattrs():
    print("%s : %s" %(at, nc_i.variables['time'].getncattr(at)))

odict_keys(['lat', 'lon', 'time', 'swh', 'swhcor', 'N'])
long_name : longitude in degrees east
units : degrees_east
add_offset : 0.0
scale_factor : 0.01
valid_range : [    0. 36000.]
long_name : latitude in degrees north
units : degrees_north
add_offset : 0.0
scale_factor : 0.01
valid_range : [-9000.  9000.]
long_name : time
units : days since 1900-1-1 0:0:0
add_offset : 0.0
scale_factor : 1.0
valid_range : [     0. 401767.]


Call Bia's Ifremer Product 1 daily binned data from the server

In [6]:
swh_i = nc_i.variables['swhcor'][:]
lon_i = nc_i.variables['lon'][:]
lat_i = nc_i.variables['lat'][:]
time_i = num2date(nc_i.variables['time'][:], nc_i.variables['time'].units) #convert time directly into datetime format instead of integer value time
nc_i.close()

Restrict the time series from 1993 to 2015 for swh_cor and deresolve to 4 degree by 4 degree grid points  

In [7]:
#find initial and final indices: 
#create year vector: 
years = np.array([y.year for y in time_i])

#creat boolean arrays and combine them: 
ind_92 = years != 1992
ind_16 = years != 2016
ind_time = ind_92*ind_16

#use the compress function to find all indices that do not lie in 1992 or 2016 and extract slices of matirx 
#along the time axis from swh
swh_i = np.compress(ind_time, swh_i, axis = 0)
time_i = time_i[ind_time] 
print(swh_i.shape,time_i.shape)

#initialize 3D array: 
swh_array_c = np.ma.masked_all([nt,nlat,nlon])

#loop through each of the matrices in the 3D array in order to deresolve each matrix one at a time:
for itime in range(0,nt,1):
    
    #call data from the 3D array 
    swh_i_day  = swh_i[itime,:,:]
    #preform the deresolution via convolution 
    swh_deres_c = running_mean(data = swh_i_day, k_dim = [4, 4], task = 'deresolve', fill_val = 'none')
    #Save running mean wsp into 3D array: 
    swh_array_c[itime,:,:] = swh_deres_c
    

(8400, 133, 360) (8400,)


Set CCMP2 filename and look at key variables and attributes

In [8]:
filename_c = sorted(glob.glob('/zdata/downloads/colosi_data_bk/binned_data/ccmpv2_wind_data/daily_binned_ccmp_v2_data/ccmp_v2_wsp_daily_binned_data_*_high_res.nc'))

#set nc variable in order to read attributes and obtained data: 
nc_c = Dataset(filename_c[0], 'r')

#print key variables:
print(nc_c.variables.keys())

#longitude
for at in nc_c.variables['lon'].ncattrs():
    print("%s : %s" %(at, nc_c.variables['lon'].getncattr(at)))

#laitude
for at in nc_c.variables['lat'].ncattrs():
    print("%s : %s" %(at, nc_c.variables['lat'].getncattr(at)))
    
#time 
for at in nc_c.variables['time'].ncattrs():
    print("%s : %s" %(at, nc_c.variables['time'].getncattr(at)))


odict_keys(['time', 'lon', 'lat', 'wsp'])
units : degrees east
units : degrees north
units : days since 1900-01-01 00:00:00
calendar : julian


Call each data set and append the data from one year onto the end of the previous year 

In [9]:
#initialize 3D array and counters/counter array
wsp_c = np.ma.masked_all([nt, nlat_c, nlon_c])
time_array = []
i = 0
yc = 0
year_c = np.array([365, 365, 365, 366, 365, 365, 365, 366, 365, 365, 365, 366, 365, 365, 365, 366, 365, 365, 365, 366, 365, 365, 365])

#restrict filename from 1993 to 2015: 
filename_c = filename_c[0:23]

#loop through each filename to call data: 
for f in filename_c: 
    
    #set nc variable in order to read attributes and obtained data: 
    nc_wnd = Dataset(f, 'r')
    #call wind speed data
    wsp = nc_wnd.variables['wsp'][:]
    time_i = num2date(nc_wnd.variables['time'][:], nc_wnd.variables['time'].units)
    #place the wsp and time data into the 3D arrays
    wsp_c[i:i+year_c[yc],:,:] = wsp
    time_array.append([time_i])
    #year counters: 
    i +=year_c[yc]
    yc += 1
    

Initialize longitude and latitude variables as well as set time as a numpy array (from a list)

In [10]:
#initialize lon and lat 
lon_c = nc_c.variables['lon'][:]
lat_c = nc_c.variables['lat'][:]

#change time to a numpy array 
time_c = np.hstack(time_array)[0]

Use the deresolve function to bring the resolution from 0.25 to 1 degree for the CCMP2 data set 

In [ ]:
#initialize 3D array: 
wsp_array_c = np.ma.masked_all([nt,nlat,nlon])

#loop through each of the matrices in the 3D array in order to deresolve each matrix one at a time:
for itime in range(0,nt,1):
    
    #call data from the 3D array 
    wsp_c_day  = wsp_c[itime,:,:]
    #preform the deresolution via convolution 
    wsp_deres_c = running_mean(data = wsp_c_day, k_dim = [16, 16], task = 'deresolve', fill_val = 'none')
    #Save running mean wsp into 3D array: 
    wsp_array_c[itime,:,:] = wsp_deres_c
    

#### The following are the coordinates of the grid boxes evaluated at each SWA region:

1) California: 

    a) North California: lon_grid => range(228,234,4)
                         lat_grid => range(98,104,4)
                         
    b) Southern California: lon_grid => range(232,238,4)
                            lat_grid => range(92,100,4)
    
    c)Western Australia: lon_grid => range(35,41,4)
                         lat_grid => range(104, 112,4)
                   
    d) South Caribbean: lon_grid => range(80,81,4)
                        lat_grid => range(280,290,4)
                        
    e) North Africa: lon_grid => range(335,340,4)
                     lat_grid => range(87,92,4)
                     
    f) South Africa (Nigeria) : lon_grid => range(7,12,4)
                                lat_grid => range(32,37,4)
                                
    g) Central West South America : lon_grid => range(280,285,4)
                                    lat_grid => range(30,36,4)
                  
    f) Arabian Sea: lon_grid => range(55,60,4)
                    lat_grid => range(71,76,4)

### WSP

Plot a two year time series of one single grid point for each SWA region 

In [ ]:
#initialize subplot axes: 
fig, axes = plt.subplots(5, 1, figsize=(14,12))
ax1, ax2, ax3, ax4, ax5 = axes.flatten()

#initialize variables:
time_ticks = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 
              'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
time = np.arange(1,721,1)
color = 'tab:blue'

#Call WSP data for two years for each SWA region
nc_ts = wsp_array_c[0:720,100,230]
wa_ts = wsp_array_c[0:720,39,108]
sc_ts = wsp_array_c[0:720,80,285]
na_ts = wsp_array_c[0:720,90,338]
sa_ts = wsp_array_c[0:720,34,10]
cwsa_ts = wsp_array_c[0:720,33,282]
as_ts = wsp_array_c[0:720,57,73]

#detrend data:
wsp_dt_nc = detrend(nc_ts)
wsp_dt_wa = detrend(wa_ts)
wsp_dt_sc = detrend(sc_ts)
wsp_dt_na = detrend(na_ts)
wsp_dt_sa = detrend(sa_ts)
wsp_dt_cwsa = detrend(cwsa_ts)
wsp_dt_as = detrend(as_ts)

#compute the running mean for WSP to cut down the noise
wsp_rm_nc = running_mean(data=wsp_dt_nc, k_dim=[4,1], task='running_mean', fill_val='mask')
wsp_rm_wa = running_mean(data=wsp_dt_wa, k_dim=[4,1], task='running_mean', fill_val='mask')
wsp_rm_sc = running_mean(data=wsp_dt_sc, k_dim=[4,1], task='running_mean', fill_val='mask')
wsp_rm_na = running_mean(data=wsp_dt_na, k_dim=[4,1], task='running_mean', fill_val='mask')
wsp_rm_sa = running_mean(data=wsp_dt_sa, k_dim=[4,1], task='running_mean', fill_val='mask')
wsp_rm_cwsa = running_mean(data=wsp_dt_cwsa, k_dim=[4,1], task='running_mean', fill_val='mask')
wsp_rm_as = running_mean(data=wsp_dt_as, k_dim=[4,1], task='running_mean', fill_val='mask')

#compute least square fit:
wsp_hfit_nc, x_swh_nc = least_square_fit(data = wsp_dt_nc, trend = 'sinusoidal', parameters = 3, 
                                         period = 365.25, fill_val = 'mask')
wsp_hfit_wa, x_swh_wa = least_square_fit(data = wsp_dt_wa, trend = 'sinusoidal', parameters = 3, 
                                         period = 365.25, fill_val = 'mask')
wsp_hfit_sc, x_swh_sc = least_square_fit(data = wsp_dt_sc, trend = 'sinusoidal', parameters = 3, 
                                         period = 365.25, fill_val = 'mask')
wsp_hfit_na, x_swh_na = least_square_fit(data = wsp_dt_na, trend = 'sinusoidal', parameters = 3, 
                                         period = 365.25, fill_val = 'mask')
wsp_hfit_sa, x_swh_sa = least_square_fit(data = wsp_dt_sa, trend = 'sinusoidal', parameters = 3, 
                                         period = 365.25, fill_val = 'mask')
wsp_hfit_cwsa, x_swh_cwsa = least_square_fit(data = wsp_dt_cwsa, trend = 'sinusoidal', parameters = 3, 
                                             period = 365.25, fill_val = 'mask')
wsp_hfit_as, x_swh_as = least_square_fit(data = wsp_dt_as, trend = 'sinusoidal', parameters = 3, 
                                         period = 365.25, fill_val = 'mask')

#compute the parameters of the least square fit  
wsp_rms, wsp_amp1, wsp_phase1_nc, wsp_amp2, wsp_phase2, wsp_cod = character_LSF(data = wsp_dt_nc, 
                                                                                model = wsp_hfit_nc, 
                                                                                x_solution = x_swh_nc, 
                                                                                trend = 'sinusoidal', 
                                                                                parameters = 3, 
                                                                                fill_val = 'mask')
#plot regional time series

############ Subplot 1 ############
#Northern California
ax1.plot(np.arange(1,len(wsp_rm_nc)+1), wsp_rm_nc, 'b-')
ax1.plot(time, wsp_dt_nc, 'b-.', alpha=0.3)
ax1.plot(time, wsp_hfit_nc, 'r', alpha=0.6 )
#set x-axis
ax1.set_xlim([0, 721])
start, end = ax1.get_xlim()
#ax1.xaxis.set_ticks(np.arange(start+1, end+1, 1))
#hind all ticklabels except the 1st of each month 
counter = 0
#for label in ax1.xaxis.get_ticklabels()[::1]:
#    counter += 1
#    if counter != 30:
#        label.set_visible(False)
#    elif counter == 30: 
#        counter = 0
#ax1.set_xticklabels(time_ticks)
#set y-axis 
ax1.set_ylabel('WSP [m]', color=color)
ax1.set_ylim([2, 15])
ax1.grid(color=color, linestyle='-.', linewidth=1, alpha = 0.2)

############ Subplot 1 ############
#west Australia
ax2.plot(np.arange(1,len(wsp_rm_wa)+1), wsp_rm_wa, 'b-')
ax2.plot(time, wsp_dt_wa, 'b-.', alpha=0.3)
ax2.plot(time, wsp_hfit_wa, 'r', alpha=0.6 )
#set x-axis
ax2.set_xlim([0, 721])
#start, end = ax2.get_xlim()
#ax2.xaxis.set_ticks(np.arange(start+1, end+1, 1))
#hind all ticklabels except the 1st of each month 
#counter = 0
#for label in ax2.xaxis.get_ticklabels()[::1]:
#    counter += 1
#    if counter != 30:
#        label.set_visible(False)
#    elif counter == 30: 
#        counter = 0
#ax2.set_xticklabels(time_ticks)
#set y-axis 
ax2.set_ylabel('WSP [m]', color=color)
ax2.set_ylim([2, 15])
ax2.grid(color=color, linestyle='-.', linewidth=1, alpha = 0.2)

############ Subplot 3 ############
#Southern Caribbean
ax3.plot(np.arange(1,len(wsp_rm_sc)+1), wsp_rm_sc, 'b-')
ax3.plot(time, wsp_dt_sc, 'b-.', alpha=0.3)
ax3.plot(time, wsp_hfit_sc, 'r', alpha=0.6 )
#set x-axis
ax3.set_xlim([0, 721])
start, end = ax3.get_xlim()
#ax1.xaxis.set_ticks(np.arange(start+1, end+1, 1))
#hind all ticklabels except the 1st of each month 
counter = 0
#for label in ax1.xaxis.get_ticklabels()[::1]:
#    counter += 1
#    if counter != 30:
#        label.set_visible(False)
#    elif counter == 30: 
#        counter = 0
#ax1.set_xticklabels(time_ticks)
#set y-axis 
ax3.set_ylabel('WSP [m]', color=color)
ax3.set_ylim([2, 15])
ax3.grid(color=color, linestyle='-.', linewidth=1, alpha = 0.2)

############ Subplot 4 ############
#North africa
ax4.plot(np.arange(1,len(wsp_rm_na)+1), wsp_rm_na, 'b-')
ax4.plot(time, wsp_dt_na, 'b-.', alpha=0.3)
ax4.plot(time, wsp_hfit_na, 'r', alpha=0.6 )
#set x-axis
ax4.set_xlim([0, 721])
start, end = ax4.get_xlim()
#ax1.xaxis.set_ticks(np.arange(start+1, end+1, 1))
#hind all ticklabels except the 1st of each month 
counter = 0
#for label in ax1.xaxis.get_ticklabels()[::1]:
#    counter += 1
#    if counter != 30:
#        label.set_visible(False)
#    elif counter == 30: 
#        counter = 0
#ax1.set_xticklabels(time_ticks)
#set y-axis 
ax4.set_ylabel('WSP [m]', color=color)
ax4.set_ylim([2, 15])
ax4.grid(color=color, linestyle='-.', linewidth=1, alpha = 0.2)

############ Subplot 5 ############
#South africa
ax5.plot(np.arange(1,len(wsp_rm_sa)+1), wsp_rm_sa, 'b-')
ax5.plot(time, wsp_dt_sa, 'b-.', alpha=0.3)
ax5.plot(time, wsp_hfit_sa, 'r', alpha=0.6 )
#set x-axis
ax5.set_xlim([0, 721])
start, end = ax5.get_xlim()
#ax1.xaxis.set_ticks(np.arange(start+1, end+1, 1))
#hind all ticklabels except the 1st of each month 
counter = 0
#for label in ax1.xaxis.get_ticklabels()[::1]:
#    counter += 1
#    if counter != 30:
#        label.set_visible(False)
#    elif counter == 30: 
#        counter = 0
#ax1.set_xticklabels(time_ticks)
#set y-axis 
ax5.set_ylabel('WSP [m]', color=color)
ax5.set_ylim([2, 15])
ax5.grid(color=color, linestyle='-.', linewidth=1, alpha = 0.2)




### Developmental Code

In [ ]:
character_LSF?

In [ ]:
time = np.arange(1,721,1)
print(time.shape)

In [ ]:
running_mean?

In [ ]:
wsp_dt_nc.shape

In [ ]:
wsp_array_c.shape

In [ ]:
360/7

In [ ]:
133/4

In [ ]:
133/3

In [ ]:
133/7